# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
file_path = "../WeatherPy/output_data/cities.csv"
cities_df = pd.read_csv(file_path)
cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nantucket,41.2835,-70.0995,24.80,68,1,3.44,US,1611624863
1,1,leh,34.1667,77.5833,-0.56,64,53,1.21,IN,1611624782
2,2,waddan,29.1614,16.1390,57.92,37,96,4.65,LY,1611625030
3,3,benito juarez,19.3850,-99.1650,66.20,42,75,14.97,MX,1611625030
4,4,nioro,13.3500,-15.7500,67.19,26,0,0.51,GM,1611624938
...,...,...,...,...,...,...,...,...,...,...
535,535,bundibugyo,0.7414,30.0417,68.25,82,87,3.53,UG,1611625109
536,536,derzhavinsk,51.1000,66.3167,24.46,98,100,10.71,KZ,1611625109
537,537,council bluffs,41.2619,-95.8608,24.80,86,90,12.66,US,1611624850
538,538,sept-iles,50.2001,-66.3821,34.00,60,100,9.22,CA,1611625023


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key=g_key)

locations = cities_df[["Lat", "Lng"]]

humidity = cities_df["Humidity"]

In [11]:
#Create Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level = 2)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity=100, point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
narrow_df = cities_df.loc[(cities_df["Wind Speed"] <= 10) & (cities_df["Cloudiness"] == 0) & \
                         (cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] <= 80)].dropna()

narrow_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
113,113,tabou,4.4230,-7.3528,77.50,87,0,6.22,CI,1611625042
127,127,non sang,16.8687,102.5664,73.99,78,0,3.44,TH,1611625043
128,128,axim,4.8699,-2.2405,77.61,82,0,5.08,GH,1611625043
197,197,harper,4.3750,-7.7169,77.67,86,0,5.97,LR,1611624790
222,222,guanica,17.9716,-66.9080,73.00,89,0,7.00,PR,1611625053
228,228,les cayes,18.2000,-73.7500,73.51,81,0,4.05,HT,1611625054
256,256,rocha,-34.4833,-54.3333,71.10,87,0,8.50,UY,1611624921
272,272,sao lourenco do sul,-31.3653,-51.9783,77.00,85,0,7.43,BR,1611624774
313,313,ratnagiri,16.9833,73.3000,72.50,54,0,4.56,IN,1611625062
329,329,saint-philippe,-21.3585,55.7679,77.00,73,0,8.05,RE,1611625064


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotel_df = narrow_df.loc[:,["City", "Country", "Lat", "Lng"]]

hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
113,tabou,CI,4.4230,-7.3528,
127,non sang,TH,16.8687,102.5664,
128,axim,GH,4.8699,-2.2405,
197,harper,LR,4.3750,-7.7169,
222,guanica,PR,17.9716,-66.9080,
228,les cayes,HT,18.2000,-73.7500,
256,rocha,UY,-34.4833,-54.3333,
272,sao lourenco do sul,BR,-31.3653,-51.9783,
313,ratnagiri,IN,16.9833,73.3000,
329,saint-philippe,RE,-21.3585,55.7679,


In [19]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [23]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response["results"]
    
    try:
        print(f"Closest Hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field...skipping.")
        
        
    print("----------------------------------------")
    
    
    
    
    
print("----------Search Complete------------------")

Retrieving Results for Index 113: tabou.
Closest Hotel in tabou is hotêl doufoulougou.
----------------------------------------
Retrieving Results for Index 127: non sang.
Closest Hotel in non sang is Sabaidee@Home Nonsang.
----------------------------------------
Retrieving Results for Index 128: axim.
Closest Hotel in axim is Ankobra Beach.
----------------------------------------
Retrieving Results for Index 197: harper.
Closest Hotel in harper is Grands' Guesthouse.
----------------------------------------
Retrieving Results for Index 222: guanica.
Closest Hotel in guanica is Parador Guánica 1929.
----------------------------------------
Retrieving Results for Index 228: les cayes.
Closest Hotel in les cayes is Villa Orphee.
----------------------------------------
Retrieving Results for Index 256: rocha.
Closest Hotel in rocha is Casa Rodo 53.
----------------------------------------
Retrieving Results for Index 272: sao lourenco do sul.
Closest Hotel in sao lourenco do sul is Pla

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
